In [6]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Optional, Dict, Any
import faiss
from sentence_transformers import SentenceTransformer
from transformers import pipeline, AutoTokenizer
import time
import torch
import json
from datasets import load_dataset
import os
import glob

In [10]:

def save_wiki_to_txt(num_docs=1000):
    """
    Wikipedia adathalmaz betöltése és szövegek mentése .txt fájlokba
    
    Args:
        num_docs (int): Mentendő dokumentumok száma (alapértelmezetten 1000)
    """
    # Mappa létrehozása
    os.makedirs('Datas', exist_ok=True)
    print("'Datas' mappa létrehozva/sikeresen ellenőrizve")
    
    try:
        # Adathalmaz betöltése
        print("Wikipedia adathalmaz betöltése...")
        dataset = load_dataset("wikipedia", "20220301.simple", trust_remote_code=True)
        print(f"Adathalmaz sikeresen betöltve. Dokumentumok száma: {len(dataset['train'])}")
        
        # Dokumentumok mentése
        saved_count = 0
        for i, doc in enumerate(dataset['train']):
            if i >= num_docs:
                break
                
            # Fájlnév generálása
            filename = f"Datas/wiki_{i+1}.txt"
            
            # Szöveg mentése
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(doc['text'])
            
            saved_count += 1
            if saved_count % 10 == 0:  # Minden 10. fájl után jelentés
                print(f"{saved_count}/{num_docs} fájl mentve...")
        
        print(f"\nSikeres mentés! {saved_count} fájl mentve a 'Datas' mappába.")
        return True
    
    except Exception as e:
        print(f"Hiba történt: {str(e)}")
        return False

In [11]:
#csak egyszer kell futtatni az elején, hogy a wiki adathalmazt elmentsük

save_wiki_to_txt()

`trust_remote_code` is not supported anymore.
Please check that the Hugging Face dataset 'wikipedia' isn't based on a loading script and remove `trust_remote_code`.
If the dataset is based on a loading script, please ask the dataset author to remove it and convert it to a standard format like Parquet.


'Datas' mappa létrehozva/sikeresen ellenőrizve
Wikipedia adathalmaz betöltése...


Using the latest cached version of the dataset since wikipedia couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration '20220301.simple' at C:\Users\User\.cache\huggingface\datasets\wikipedia\20220301.simple\2.0.0\d41137e149b2ea90eead07e7e3f805119a8c22dd1d5b61651af8e3e3ee736001 (last modified on Sat Aug  9 15:31:54 2025).


Adathalmaz sikeresen betöltve. Dokumentumok száma: 205328
10/1000 fájl mentve...
20/1000 fájl mentve...
30/1000 fájl mentve...
40/1000 fájl mentve...
50/1000 fájl mentve...
60/1000 fájl mentve...
70/1000 fájl mentve...
80/1000 fájl mentve...
90/1000 fájl mentve...
100/1000 fájl mentve...
110/1000 fájl mentve...
120/1000 fájl mentve...
130/1000 fájl mentve...
140/1000 fájl mentve...
150/1000 fájl mentve...
160/1000 fájl mentve...
170/1000 fájl mentve...
180/1000 fájl mentve...
190/1000 fájl mentve...
200/1000 fájl mentve...
210/1000 fájl mentve...
220/1000 fájl mentve...
230/1000 fájl mentve...
240/1000 fájl mentve...
250/1000 fájl mentve...
260/1000 fájl mentve...
270/1000 fájl mentve...
280/1000 fájl mentve...
290/1000 fájl mentve...
300/1000 fájl mentve...
310/1000 fájl mentve...
320/1000 fájl mentve...
330/1000 fájl mentve...
340/1000 fájl mentve...
350/1000 fájl mentve...
360/1000 fájl mentve...
370/1000 fájl mentve...
380/1000 fájl mentve...
390/1000 fájl mentve...
400/1000 fájl m

True

In [52]:
#classok

class AgentState(TypedDict):
    question: str
    decomposed_questions: List[str]  # Részfeladatok
    context: List[Dict[str, Any]]    # Retrieval eredmények
    sub_answers: List[Dict[str, Any]] # Részválaszok
    final_answer: str                # Végső válasz

class VectorStore:
    """FAISS alapú vektor tár"""
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.embedder = SentenceTransformer(model_name)
        self.index = None
        self.chunks = []
        
    def build_index(self, chunks: List[Dict]):
        self.chunks = chunks
        texts = [c['text'] for c in chunks]
        embeddings = self.embedder.encode(texts, convert_to_numpy=True)
        dim = embeddings.shape[1]
        self.index = faiss.IndexFlatIP(dim)
        faiss.normalize_L2(embeddings)
        self.index.add(embeddings)
        print(f"Index építve: {len(chunks)} dokumentummal")
        
    def retrieve(self, query: str, k: int = 3) -> List[Dict]:
        if not self.index:
            raise ValueError("Index nincs inicializálva")
        q_emb = self.embedder.encode([query], convert_to_numpy=True)
        faiss.normalize_L2(q_emb)
        k = min(k, len(self.chunks))
        D, I = self.index.search(q_emb, k)
        return [self.chunks[i] for i in I[0] if i < len(self.chunks)]

class GenerationModel:
    """Generatív modell burkoló"""
    def __init__(self, model_name='gpt2'):
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        tokenizer.pad_token = tokenizer.eos_token
        self.generator = pipeline(
            'text-generation', 
            model=model_name,
            device_map='auto',
            tokenizer=tokenizer
        )
        
    def generate(self, prompt: str, max_length=150) -> str:
        start = time.time()
        output = self.generator(
            prompt,
            max_new_tokens=max_length,
            do_sample=True,
            temperature=0.3,
            top_p=0.9,
            repetition_penalty=2.0,  # Ismétlődés csökkentése
            num_return_sequences=1,
            no_repeat_ngram_size=3  # N-gram ismétlődés tiltása
        )
        latency = time.time() - start
        print(f"Generálás kész ({latency:.2f}s)")
        
        full_text = output[0]['generated_text']

        if "Válasz:" in full_text:
            return full_text.split("Válasz:")[-1].strip()
        return output[0]['generated_text'].replace(prompt, "", 1).strip()

In [53]:
#függvények

# 3. Graph csomópontok definíciója
def decompose_node(state: AgentState) -> dict:
    """Feladat lebontása részfeladatokra"""
    print("\n" + "="*50)
    print("1. FELADAT LEBONTÁS")
    print("="*50)
    
    # Heurisztikus feladatbontás
    if "," in state["question"]:
        sub_questions = [q.strip() for q in state["question"].split(",")]
    elif "?" in state["question"]:
        sub_questions = [q.strip() + "?" for q in state["question"].split("?") if q]
    else:
        words = state["question"].split()
        if len(words) > 8:
            mid = len(words) // 2
            sub_questions = [" ".join(words[:mid]), " ".join(words[mid:])]
        else:
            sub_questions = [state["question"]]
    
    print(f"Összetett kérdés lebontva {len(sub_questions)} részre:")
    for i, q in enumerate(sub_questions, 1):
        print(f"  [{i}] {q}")
    
    return {"decomposed_questions": sub_questions}

def retrieve_node(state: AgentState, vector_store: VectorStore) -> dict:
    """Releváns kontextus keresése minden részfeladathoz"""
    print("\n" + "="*50)
    print("2. KONTEXTUS KERESÉS")
    print("="*50)
    
    all_retrieved = []
    for q in state["decomposed_questions"]:
        print(f"\nKérdés: '{q}'")
        results = vector_store.retrieve(q, k=3)
        print(f"Találatok: {len(results)} dokumentum")
        for i, doc in enumerate(results, 1):
            print(f"  - Doc {i}: {doc['text'][:80]}... (ID: {doc['id']})")
        all_retrieved.extend(results)
    
    # Duplikátumok eltávolítása
    unique_docs = {doc['id']: doc for doc in all_retrieved}.values()
    return {"context": list(unique_docs)}

def generate_node(state: AgentState, gen_model: GenerationModel) -> dict:
    """Válasz generálása részfeladatokra"""
    print("\n" + "="*50)
    print("3. VÁLASZ GENERÁLÁS")
    print("="*50)
    
    sub_answers = []

    truncated_context = [
        {"id": doc["id"], "text": doc["text"][:500] + "..." if len(doc["text"]) > 500 else doc["text"]}
        for doc in state["context"]
    ]

    context_str = "\n".join([doc['text'] for doc in truncated_context])
    
    for i, q in enumerate(state["decomposed_questions"], 1):
        print(f"\nGenerálás [{i}/{len(state['decomposed_questions'])}]: {q}")
        
        prompt = (
            f"A következő információk alapján válaszolj a kérdésre:\n{context_str}\n\n"
            f"Kérdés: {q}"
        )
        
        answer = gen_model.generate(prompt)
        print(f"  Generált válasz: {answer[:120]}" + ("..." if len(answer) > 120 else ""))
        
        sub_answers.append({
            "subquestion": q,
            "answer": answer,
            "context_ids": [doc['id'] for doc in state["context"]]
        })
    
    return {"sub_answers": sub_answers}

def aggregate_node(state: AgentState) -> dict:
    """Részválaszok összeállítása végső válasszá"""
    print("\n" + "="*50)
    print("4. VÉGLEGES VÁLASZ")
    print("="*50)
    
    final_answer = "\n\n".join(
        f"{sa['answer']}" 
        for sa in state["sub_answers"]
    )
    
    print(f"Végső válasz készült ({len(state['sub_answers'])} részfeladatból)")
    return {"final_answer": final_answer}

# 4. Graph összeállítása
def create_workflow(vector_store: VectorStore, gen_model: GenerationModel) -> StateGraph:
    workflow = StateGraph(AgentState)
    
    # Csomópontok hozzáadása
    workflow.add_node("decompose", decompose_node)
    workflow.add_node("retrieve", lambda state: retrieve_node(state, vector_store))
    workflow.add_node("generate", lambda state: generate_node(state, gen_model))
    workflow.add_node("aggregate", aggregate_node)
    
    # Élek definiálása
    workflow.set_entry_point("decompose")
    workflow.add_edge("decompose", "retrieve")
    workflow.add_edge("retrieve", "generate")
    workflow.add_edge("generate", "aggregate")
    workflow.add_edge("aggregate", END)
    
    return workflow.compile()

def create_demo_data() -> List[Dict]:
    """Demó adatbázis létrehozása a 'Datas' mappában lévő txt fájlokból"""
    datas = []
    data_dir = "Datas"
    
    # Ellenőrizzük, hogy a mappa létezik-e
    if not os.path.exists(data_dir):
        print(f"Figyelem: A '{data_dir}' mappa nem létezik!")
        return datas
    
    # Összes .txt fájl listázása a mappában
    txt_files = glob.glob(os.path.join(data_dir, "*.txt"))
    
    # Ha nincsenek fájlok
    if not txt_files:
        print(f"Figyelem: A '{data_dir}' mappában nincsenek .txt fájlok!")
        return datas
    
    # Fájlok beolvasása
    for file_path in txt_files:
        try:
            # Fájlnév kinyerése (kiterjesztés nélkül)
            file_name = os.path.basename(file_path)
            doc_id = os.path.splitext(file_name)[0]  # Kiterjesztés eltávolítása
            
            # Tartalom beolvasása
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                
                # Csak ha van tartalom
                if content.strip():
                    datas.append({
                        'id': doc_id,
                        'text': content
                    })
                    
        except Exception as e:
            print(f"Hiba a(z) '{file_path}' fájl feldolgozása közben: {str(e)}")
    
    print(f"{len(datas)} dokumentum betöltve a '{data_dir}' mappából")
    return datas


In [54]:
#ide jön a működésnek a többi része majd, ami demózza a chatbot működését


print("="*80)
print("AGENTIC RAG PROTOTÍPUS - LANGGRAPH IMPLEMENTÁCIÓ")
print("="*80 + "\n")
    
# Komponensek inicializálása
vector_store = VectorStore()
gen_model = GenerationModel()
chunks = create_demo_data()
print(chunks[:3])  # Első 3 dokumentum kiírása ellenőrzéshez
vector_store.build_index(chunks)
    
# Workflow összeállítása
workflow = create_workflow(vector_store, gen_model)
    
# Példa kérdés
question = (
    "When is Nobel Prize Day in December? And tell me something about December on the whole."
)
    
# Állapot inicializálása
init_state = AgentState(
    question=question,
    decomposed_questions=[],
    context=[],
    sub_answers=[],
    final_answer=""
)
    
# Workflow futtatása
print(f"\nKezdő kérdés: '{question}'\n")
result = workflow.invoke(init_state)
    
# Végeredmény megjelenítése
print("\n" + "="*80)
print("VÉGEREDMÉNY")
print("="*80)
print(result["final_answer"])
    
# Debug információk
print("\n" + "="*80)
print("DEBUG INFORMÁCIÓ")
print("="*80)
print(f"Felhasznált dokumentumok: {len(result['context'])}")
print(f"Részfeladatok: {len(result['decomposed_questions'])}")
print(f"Generálások: {len(result['sub_answers'])}")

AGENTIC RAG PROTOTÍPUS - LANGGRAPH IMPLEMENTÁCIÓ



Device set to use cpu


1000 dokumentum betöltve a 'Datas' mappából
[{'id': 'wiki_1', 'text': 'April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril\'s flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other\'s last days are exactly 35 weeks (245 days) apart.\n\nIn common years, April sta